# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the city_data.csv file from output folder 
ct_df = pd.read_csv("../output_data/city_data.csv")
ct_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Aripuanã,-9.1667,-60.6333,85.60,59,70,1.92,BR,1610455845
1,Jamestown,42.0970,-79.2353,27.00,80,90,6.91,US,1610455581
2,Bluff,-46.6000,168.3333,52.00,84,16,1.01,NZ,1610455847
3,Narsaq,60.9167,-46.0500,42.80,33,75,20.71,GL,1610455848
4,Hong Kong,22.2855,114.1577,55.99,30,37,11.16,HK,1610455586
...,...,...,...,...,...,...,...,...,...
693,Muskegon,43.3000,-86.1334,35.60,80,90,14.97,US,1610456157
694,Mahina,-17.5065,-149.4890,77.00,78,75,3.44,PF,1610456255
695,Kushiro,42.9750,144.3747,23.00,93,75,10.36,JP,1610456255
696,Haines Junction,60.7522,-137.5108,8.60,92,99,1.14,CA,1610456256


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = ct_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = ct_df["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(20, 150), zoom_level=1.6)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,dissipating=False, max_intensity=max(humidity), point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities to fit ideal weather:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Humidity is lower than 80% but greater than 50.
select_city_df = ct_df.loc[(ct_df["Max Temp"] >= 70) & (ct_df["Max Temp"] <= 80) &\
                           (ct_df["Wind Speed"] <= 10) & (ct_df["Cloudiness"] == 0) &\
                          (ct_df["Humidity"] >= 50) & (ct_df["Humidity"] <= 80)]

# Drop any rows that don't contain all above conditions
select_city_df = select_city_df.dropna()
select_city_df.reset_index(drop=True)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Puerto Escondido,15.8500,-97.0667,72.73,54,0,5.28,MX,1610455988
1,Diu,20.7141,70.9822,75.74,52,0,3.47,IN,1610456127
2,Barra de Tecoanapa,16.5167,-98.7500,72.10,77,0,4.25,MX,1610456149
3,Dubbo,-32.2500,148.6167,71.60,53,0,3.44,AU,1610456189


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = select_city_df.reset_index(drop=True)

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Puerto Escondido,15.8500,-97.0667,72.73,54,0,5.28,MX,1610455988,
1,Diu,20.7141,70.9822,75.74,52,0,3.47,IN,1610456127,
2,Barra de Tecoanapa,16.5167,-98.7500,72.10,77,0,4.25,MX,1610456149,
3,Dubbo,-32.2500,148.6167,71.60,53,0,3.44,AU,1610456189,


In [7]:
# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

# Loop through the cities_pd and run a hotel search for each city
for index, row in hotel_df.iterrows():
# Extract latitude and longitude and the city name
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]    
    
    # update address key value
    params["location"] = f"{lat},{lng}"
    
    # make request
    response = requests.get(base_url, params=params).json()
    
    # test if there is a an available hotel in the city
    try:
        hotel_name = response["results"][0]["name"]
        print(f"Closest hotel in {city_name} is {hotel_name}.")
        hotel_df.loc[index, "Hotel Name"] = hotel_name

    # if there is no hotel available, throw an error message
    except (KeyError, IndexError):
        print(f"No Hotel available in {city_name}")

Closest hotel in Puerto Escondido is Selina Puerto Escondido.
Closest hotel in Diu is Hotel TGH - The Grand Highness.
Closest hotel in Barra de Tecoanapa is Hotel Garapacho.
Closest hotel in Dubbo is Quest Dubbo.


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))